# Banking and Unemployment
---

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import time
from census import Census
from us import states

# census key
from config import (census_key, gkey)

# Census API Key
c = Census(census_key, year=2013)

## Data Retrieval

In [2]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("B01003_001E", "B23025_005E"), {
                         'for': 'zip code tabulation area:*'})

# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B23025_005E": "Unemployment Count",
                                      "zip code tabulation area": "Zipcode"})

# Add in Employment Rate (Unemployment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["Population", "Unemployment Rate", "Zipcode"]]

# Visualize
print(len(census_pd))
census_pd.head()

33120


,Population,Unemployment Rate,Zipcode
0,18450.0,7.934959,00601
1,41302.0,11.541814,00602
2,53683.0,8.086359,00603
3,6591.0,2.033075,00606
4,28963.0,3.915340,00610


## Combine Data

In [3]:
# Import the original data we analyzed earlier. Use dtype="object" to match other
census_data_original = pd.read_csv(
    "../Resources/zip_bank_data.csv", dtype="object", encoding="utf-8")

# Visualize
print(len(census_data_original))
census_data_original.head()

700


,Zipcode,Address,Population,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777,22137,13.18375282,42.6306916,-85.2929384,9
1,71405,"Ball, LA 71405, USA",5687,35.8,55242,23941,15.73764727,31.4061799,-92.396174,12
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944,35961,9.295946358,38.2201614,-76.4967919,9
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125,18884,24.40944882,35.7310368,-87.2419299,0
4,58466,"Marion, ND 58466, USA",509,44.5,69844,36981,9.430255403,46.5594224,-98.3481542,1


In [4]:
# Merge the two data sets along zip code
zipcode_merge = pd.merge(census_data_original, census_pd, how='left', on=["Zipcode", "Zipcode"])
zipcode_merge.head()

# Save the revised Data Frame as a csv

zipcode_merge.to_csv("../Resources/bank_data_with_employment_om.csv", encoding="utf-8", index=False)

# Visualize
print(len(zipcode_merge))
zipcode_merge.head()


700


,Zipcode,Address,Population_x,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count,Population_y,Unemployment Rate
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777,22137,13.18375282,42.6306916,-85.2929384,9,19031.0,5.312385
1,71405,"Ball, LA 71405, USA",5687,35.8,55242,23941,15.73764727,31.4061799,-92.396174,12,5687.0,4.571830
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944,35961,9.295946358,38.2201614,-76.4967919,9,6562.0,4.053642
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125,18884,24.40944882,35.7310368,-87.2419299,0,762.0,10.236220
4,58466,"Marion, ND 58466, USA",509,44.5,69844,36981,9.430255403,46.5594224,-98.3481542,1,509.0,1.178782


## Heatmap of poverty rate

In [5]:
# Configure gmaps with API key
import gmaps
gmaps.configure(api_key=gkey)

In [10]:
# Store 'Lat' and 'Lng' into  locations 
locations = zipcode_merge[["Lat", "Lng"]].astype(float)



# Convert Poverty Rate to float and store
poverty_rate = zipcode_merge["Poverty Rate"].astype(float)


# HINT: be sure to handle NaN values


In [11]:
zipcode_merge.dropna()

,Zipcode,Address,Population_x,Median Age,Household Income,Per Capita Income,Poverty Rate,Lat,Lng,Bank Count,Population_y,Unemployment Rate
0,49058,"Hastings, MI 49058, USA",19031,40.8,46777,22137,13.18375282,42.6306916,-85.2929384,9,19031.0,5.312385
1,71405,"Ball, LA 71405, USA",5687,35.8,55242,23941,15.73764727,31.4061799,-92.396174,12,5687.0,4.571830
2,20634,"Great Mills, MD 20634, USA",6562,31.9,79944,35961,9.295946358,38.2201614,-76.4967919,9,6562.0,4.053642
3,38487,"Williamsport, TN 38487, USA",762,41.6,38125,18884,24.40944882,35.7310368,-87.2419299,0,762.0,10.236220
4,58466,"Marion, ND 58466, USA",509,44.5,69844,36981,9.430255403,46.5594224,-98.3481542,1,509.0,1.178782
5,89316,"Eureka, NV 89316, USA",1513,38.7,63603,26280,14.87111699,39.564892,-115.9940986,1,1513.0,0.660939
6,66507,"Maple Hill, KS 66507, USA",1237,31.8,67500,23114,2.425222312,38.9793352,-96.0044121,1,1237.0,1.535974
7,93637,"Madera, CA 93637, USA",40515,29.5,48974,17449,20.05183265,36.9152938,-120.2051096,0,40515.0,3.258053
8,92392,"Victorville, CA 92392, USA",54557,27.9,58667,19090,19.85446414,34.4854334,-117.4203974,14,54557.0,7.027513
9,49080,"Plainwell, MI 49080, USA",15624,40.9,53743,25048,11.97516641,42.446414,-85.5817911,6,15624.0,6.003584


In [12]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=poverty_rate, dissipating=False)
# Add layer
heat_layer.dissipating = False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
# Convert bank rate to list
bank_rate = zipcode_merge["Bank Count"].tolist()

In [14]:
# Create bank symbol layer
bank_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Bank amount: {bank}" for bank in bank_rate]
)


fig = gmaps.figure()
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Create a combined map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(bank_layer)

fig

Figure(layout=FigureLayout(height='420px'))